In [1]:
import os

os.environ["RAY_ADDRESS"] = "10.184.206.10:7795"

In [2]:
import ray

ray.init()

2025-07-15 16:31:47,275	INFO worker.py:1588 -- Using address 10.184.206.10:7795 set in the environment variable RAY_ADDRESS
2025-07-15 16:31:47,287	INFO worker.py:1723 -- Connecting to existing Ray cluster at address: 10.184.206.10:7795...
2025-07-15 16:31:47,298	INFO worker.py:1908 -- Connected to Ray cluster. View the dashboard at http://10.184.206.10:7796 


Python version:,3.12.11
Ray version:,2.47.0
Dashboard:,http://10.184.206.10:7796


(Stage 00 - FilePartitioningStage pid=2904609) 2025-07-15 16:32:32.169 | INFO     | cosmos_xenna.ray_utils.stage_worker:setup:422 - Setting up actor for stage=Stage 00 - FilePartitioningStage
(Stage 00 - FilePartitioningStage pid=2904609) 2025-07-15 16:32:32.169 | INFO     | cosmos_xenna.ray_utils.stage_worker:setup:427 - Finished setting up actor for stage=Stage 00 - FilePartitioningStage
(Stage 04 - JsonlWriter pid=2904611) 2025-07-15 16:32:32.141 | INFO     | cosmos_xenna.ray_utils.stage_worker:setup_on_node:389 - Setting up actor for stage=Stage 04 - JsonlWriter on node=4d2fa3e2a9deca2338c41cdc38a8a974112388c551b703fecc2de6e9
(Stage 04 - JsonlWriter pid=2904611) 2025-07-15 16:32:32.141 | INFO     | cosmos_xenna.ray_utils.stage_worker:setup_on_node:392 - Finished setting up actor for stage=Stage 04 - JsonlWriter on node=4d2fa3e2a9deca2338c41cdc38a8a974112388c551b703fecc2de6e9
(Stage 04 - JsonlWriter pid=2904611) 2025-07-15 16:32:32.151 | INFO     | cosmos_xenna.ray_utils.stage_worke

In [3]:
from ray_curator.pipeline import Pipeline
from ray_curator.stages.classifiers.hf_model import HFQualityClassifier
from ray_curator.stages.classifiers.hf_tokenizer import HFTokenizer
from ray_curator.stages.io.reader import JsonlReader
from ray_curator.stages.io.writer import JsonlWriter
from ray_curator.stages.resources import Resources

In [4]:
model_name = "nvidia/quality-classifier-deberta"
pipeline = Pipeline(
    name="tokenizer_embedder",
    stages=[
        JsonlReader("/raid/prospector-lm/clean/tinystories_train/", files_per_partition=1),
        HFTokenizer(
            id_col="id",
            text_col="text",
            model_name=model_name,
            max_seq_length=None,
            padding_side="right",
            return_text=True,
            sort_by_length=True,
            verbose=True,
        ),
        HFQualityClassifier(
            id_col="id",
            output_col="quality_pred",
            prob_col="quality_prob",
            keep_cols=["id", "text"],
            has_seq_order=True,
            micro_batch_size=256,
            verbose=True,
            max_chars=6000,
        ),
        JsonlWriter("/raid/praateekm/NeMo-Curator/ray-curator/notebooks/outputs_xenna_quality/"),
    ],
)

In [ ]:
%%time
from ray_curator.backends.experimental.ray_data import RayDataExecutor

output = pipeline.run(RayDataExecutor())

In [ ]:
%%time
from ray_curator.backends.xenna import XennaExecutor

output = pipeline.run(XennaExecutor())

2025-07-15 16:32:30.614 | INFO     | ray_curator.pipeline.pipeline:build:58 - Planning pipeline: tokenizer_embedder
2025-07-15 16:32:30.615 | INFO     | ray_curator.pipeline.pipeline:_decompose_stages:94 - Decomposing composite stage: jsonl_reader
2025-07-15 16:32:30.615 | INFO     | ray_curator.pipeline.pipeline:_decompose_stages:108 - Expanded 'jsonl_reader' into 2 execution stages
2025-07-15 16:32:30.616 | INFO     | ray_curator.backends.xenna.executor:execute:113 - Execution mode: STREAMING
2025-07-15 16:32:30.616 | INFO     | ray_curator.backends.xenna.adapter:required_resources:31 - Resources: Resources(cpus=0.5, gpu_memory_gb=0.0, nvdecs=0, nvencs=0, entire_gpu=False, gpus=0.0)
2025-07-15 16:32:30.616 | INFO     | ray_curator.backends.xenna.adapter:required_resources:31 - Resources: Resources(cpus=1.0, gpu_memory_gb=0.0, nvdecs=0, nvencs=0, entire_gpu=False, gpus=0.0)
2025-07-15 16:32:30.617 | INFO     | ray_curator.backends.xenna.adapter:required_resources:31 - Resources: Resou

In [11]:
output[0]._stage_perf

[StagePerfStats(stage_name='jsonl_reader', process_time=0.10754203796386719, actor_idle_time=0.0, input_data_size_mb=9.5367431640625e-07, num_items_processed=1),
 StagePerfStats(stage_name='hf_tokenizer', process_time=3.805401086807251, actor_idle_time=0.0, input_data_size_mb=0.0095367431640625, num_items_processed=10000),
 StagePerfStats(stage_name='model', process_time=21.417380809783936, actor_idle_time=0.0, input_data_size_mb=0.0095367431640625, num_items_processed=10000),
 StagePerfStats(stage_name='jsonl_writer', process_time=0.15916919708251953, actor_idle_time=0.0, input_data_size_mb=0.0095367431640625, num_items_processed=10000)]

In [23]:
from collections import defaultdict
import pandas as pd

name_to_perf = defaultdict(list)
for task in output:
    for stage_perf in task._stage_perf:
        name_to_perf[stage_perf.stage_name].append(stage_perf.num_items_processed / stage_perf.process_time )
        

In [24]:
pd.DataFrame(name_to_perf).describe()

,jsonl_reader,hf_tokenizer,model,jsonl_writer
count,212.000000,212.000000,212.000000,2.120000e+02
mean,11.028030,3244.299101,706.356470,1.193339e+05
std,2.544662,253.257477,56.118121,4.832887e+05
min,4.832715,2143.096890,430.644954,4.256003e+04
25%,8.988751,3095.474025,685.711762,7.396904e+04
50%,11.114963,3254.150232,714.739483,9.144077e+04
75%,13.359839,3428.047393,740.217408,9.763789e+04
max,15.312150,3811.175312,814.909375,7.119851e+06


In [27]:
pd.read_json(output[0].data[0], lines=True)

,id,text,domain_pred,domain_prob
0,TinyStories-train.txt-0,"One day, a little girl named Lily found a need...",Hobbies_and_Leisure,"[1.049e-07, 8.851e-07, 3.035e-07, 3.439e-07, 5..."
1,TinyStories-train.txt-1,"Once upon a time, there was a little car named...",Autos_and_Vehicles,"[0.0001168301, 0.00018963390000000001, 0.95690..."
2,TinyStories-train.txt-2,"One day, a little fish named Fin was swimming ...",Pets_and_Animals,"[1.42e-08, 2.6500000000000002e-08, 7.000000000..."
3,TinyStories-train.txt-3,"Once upon a time, in a land full of trees, the...",Food_and_Drink,"[8.53e-08, 3.733e-07, 2.64e-08, 2.282000000000..."
4,TinyStories-train.txt-4,"Once upon a time, there was a little girl name...",Games,"[3.83232e-05, 0.000140729, 1.17337e-05, 1.2588..."
...,...,...,...,...
9995,TinyStories-train.txt-9995,"Once upon a time, there was a graceful armcha...",Home_and_Garden,"[5.64e-08, 1.544e-07, 1.581e-07, 1.473e-07, 5...."
9996,TinyStories-train.txt-9996,Once there was a happy lady who loved carryin...,Pets_and_Animals,"[3.269e-07, 2.953e-07, 1.28e-07, 2.062e-07, 3...."
9997,TinyStories-train.txt-9997,"Once there was a little girl, Maria, who was ...",Arts_and_Entertainment,"[0.0007223269, 0.9306239486000001, 8.762820000..."
9998,TinyStories-train.txt-9998,"One day, two sisters Anna and Bella were play...",People_and_Society,"[0.0027175387, 0.3205346465, 0.0002145235, 0.0..."
